**Portrait Segmentation With Slim-Net and Mediapipe**

Mediapipe is an **open-source** framework for developing machine learning applications in **mobile**, desktop, web and IoT devices. Portrait segmentation refers to the process of segmenting a **person** in an image from its background.Here, we will develop an **android application** for portrait segmentaion using mediapipe and **tflite**.

**1. Checkout MediaPipe Github Repository**

The mediapipe **repository** contains many demo applications for android. We will modify the hair_segmentaion application, which contains the basic pipeline for **video segmentaion**.



In [ ]:
!git clone https://github.com/google/mediapipe.git

**2. Installing Bazel on Ubuntu**

Setup **baze**l on your system to build and deploy the **android** application.

In [ ]:
!sudo apt install curl gnupg
!curl -fsSL https://bazel.build/bazel-release.pub.gpg | gpg --dearmor > bazel.gpg
!sudo mv bazel.gpg /etc/apt/trusted.gpg.d/
!echo "deb [arch=amd64] https://storage.googleapis.com/bazel-apt stable jdk1.8" | sudo tee /etc/apt/sources.list.d/bazel.list
!sudo apt update && sudo apt install bazel

**3. Install a JDK (optional)**

Sometimes the **default jdk** version may cause an **error** during android sdk installation in ubuntu. So, install an older version of **openjdk-8** and configure the same as the default version of the system.

In [ ]:
!sudo apt install openjdk-8-jdk
!sudo update-alternatives --config java  # Choose OpenJDK 8
!java -version

**4. Install MediaPipe Without Android Studio (SDK & NDK)**

If the android studio is not installed in your system, you can configute mediapipe with **sdk and ndk** by running this script.

In [ ]:
!bash mediapipe/setup_android_sdk_and_ndk.sh

**Note:** If Android SDK and NDK are already installed, set **ANDROID_HOME** and **ANDROID_NDK_HOME** paths accordingly.

```
export ANDROID_HOME=<path to the Android SDK>
export ANDROID_NDK_HOME=<path to the Android NDK>
```







**5. Generate Slim-Net Tflite**

Install **tf 1.15** for converting the trained model to tflite fromat.

In [ ]:
!pip install tensorflow-gpu==1.15 # Restart kernel, if required

Load the trained keras model, and convert it into **tflite** format.

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model, Model

def bilinear_resize(x, rsize):
  return tf.image.resize_bilinear(x, [rsize,rsize], align_corners=True)

converter = tf.lite.TFLiteConverter.from_keras_model_file('/content/slim-net-157-0.02.hdf5')
tflite_model = converter.convert()
open("portrait_segmentation.tflite", "wb").write(tflite_model)

**6. Modify The Hair Segmentaion Mediapipe Application**


Set current directory to **mediapipe** and copy the **tflite** file to models directory.

In [ ]:
%cd mediapipe 
!cp /content/portrait_segmentation.tflite mediapipe/models/

**A.** Create a new directory called **portrait_segmentation** under **graphs** subdirectory and copy all the files from **hair_segmentation**.

In [ ]:
!mkdir mediapipe/graphs/portrait_segmentation
!cp -r mediapipe/graphs/hair_segmentation/* mediapipe/graphs/portrait_segmentation
!mv mediapipe/graphs/portrait_segmentation/hair_segmentation_mobile_gpu.pbtxt mediapipe/graphs/portrait_segmentation/portrait_segmentation_mobile_gpu.pbtxt

Rename the pbtxt file(mv command) as **portrait_segmentation_mobile_gpu.pbtxt** and modify the following lines :-

1. **Number of channels**: 'max_num_channels: 4' in **TfLiteConverterCalculator** to max_num_channels: 3
2. **Model name**: 'hair_segmentaion.tflite' in **TfLiteInferenceCalculator** to portrait_segmentation.tflite
3.  **Mask index**: 'output_layer_index: 1' in **TfLiteTensorsToSegmentationCalculator** to output_layer_index: 0

**B**. Add two new nodes for **luminance and sobel edge detection** and connect them to the 'throtteld_input_video' as shown below:

```
# Liminance calculator (RGB)
node {
  calculator: "LuminanceCalculator"
  input_stream: "throttled_input_video"
  output_stream: "luma_video"
}

# Sobel edge calculator (RGB)
node {
  calculator: "SobelEdgesCalculator"
  input_stream: "luma_video"
  output_stream: "sobel_video"
}
```

**C.** Now remove the 'RecolorCalculator' node, add **MaskOverlayCalculator**  and connect their inputs as shown below:

```
# Mask overlay calculator (RGB)
node {
  calculator: "MaskOverlayCalculator"
  input_stream: "VIDEO:0:throttled_input_video"
  input_stream: "VIDEO:1:sobel_video"
  input_stream: "MASK:hair_mask"
  output_stream: "OUTPUT:output_video"
}
```
**Note:** The idea is to combine the foreground in the image with background using mask, such that the filter is applied only on the background. The '**MaskOverlayCalculator**' nodes combines two input video frames using the mask  such that when mask is 0, **VIDEO:0** will be used and when mask is 1, **VIDEO:1** will be used.Remaining intermediate values (if any) will blend accordingly.

**D**. Now, inside the **BUILD** file in this directory(portrait_segmentation), change the graph name to "**portrait_segmentation_mobile_gpu.pbtxt**".

Also add the  **calculator files** inside the **cc_library** section for mobile_calculaotrs as follows:-

```
"//mediapipe/calculators/image:luminance_calculator",
"//mediapipe/calculators/image:sobel_edges_calculator",
"//mediapipe/calculators/image:mask_overlay_calculator",
```

**E**. Similary, create a new folder called '**portraitsegmentationgpu**' inside example directory at location: '**/mediapipe/examples/android/src/java/com/google/mediapipe/apps**'. 

In [ ]:
!mkdir mediapipe/examples/android/src/java/com/google/mediapipe/apps/portraitsegmentationgpu
!cp -r mediapipe/examples/android/src/java/com/google/mediapipe/apps/hairsegmentationgpu/* mediapipe/examples/android/src/java/com/google/mediapipe/apps/portraitsegmentationgpu

Finally, copy the **BUILD** file from hairsegmentationgpu folder into this new folder and **modify** the following lines:-

1. Change the name 'hairsegmentationgpu' to **portraitsegmentationgpu** everywhere.
2. Change the name 'hair_segmentaion' to **portrait_segmentation** everywhere.
3. Change the app name from 'Hair Segmentation' to **Portait Segmentation**.

See the final pbtxt file:      [ portrait_segmentation_mobile_gpu.pbtxt](/content/mediapipe/mediapipe/graphs/portrait_segmentation/portrait_segmentation_mobile_gpu.pbtxt)

**7. Build The Android Application Using Bazel**

Build the **portraitsegmentationgpu** android  application for **arm_64** architecture.

In [ ]:
!bazel build -c opt --config=android_arm64 mediapipe/examples/android/src/java/com/google/mediapipe/apps/portraitsegmentationgpu:portraitsegmentationgpu

**Note:** It may take around 20 minutes to complete the build process, for the first time.

Download or save the **apk file** from 'bazel-bin' directory in google drive.

In [ ]:
!cp bazel-bin/mediapipe/examples/android/src/java/com/google/mediapipe/apps/portraitsegmentationgpu/portraitsegmentationgpu.apk /content/drive/My\ Drive/mediapipe/